## Import library

In [1]:
import datetime 
from datetime import timedelta 
import os
import glob
import numpy as np
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import roc_curve
from torch import nn, optim

import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from itertools import cycle
from tqdm.notebook import tqdm
import re
from sklearn.metrics import roc_auc_score, plot_roc_curve

import matplotlib.pyplot as plt
import seaborn
from collections import defaultdict

from sklearn import svm

## Data load

In [2]:
# df_israel_covid 이스라엘 데이터셋
# corona_isolation_per_day_ver_0059 일일 코로나 격리자 수 
# time_series_covid19_confirmed_global 누적 확진자 수 

base_path = './opendataset'
df_israel_covid = pd.read_csv(f'{base_path}/israel_covid_small.csv')
df_israel_isolation_per_day = pd.read_csv(f'{base_path}/corona_isolation_per_day_ver_0069.csv')
df_timeseries_confirm = pd.read_csv(f'{base_path}/time_series_covid19_confirmed_global.csv')

In [3]:
df_timeseries_confirm.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/28/21,8/29/21,8/30/21,8/31/21,9/1/21,9/2/21,9/3/21,9/4/21,9/5/21,9/6/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,152960,152960,153148,153220,153260,153306,153375,153375,153375,153534
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,144079,144847,145333,146387,147369,148222,149117,150101,150997,151499
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,194671,195162,195574,196080,196527,196915,197308,197659,198004,198313
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,15025,15025,15032,15033,15046,15052,15055,15055,15055,15069
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,47079,47168,47331,47544,47781,48004,48261,48475,48656,48790


## preprocessing

In [4]:
# 누적 확진자수를 2021-08-17 와 같은 포멧으로 전처리해주는 과정

rows = []
for idx, values in enumerate(df_timeseries_confirm[df_timeseries_confirm['Country/Region'] == 'Israel'].iteritems()):
    if idx < 4: continue
    year = f"20{values[0][-2:]}"
    month = values[0][:values[0].index('/')]
    month = f'0{month}' if len(month) == 1 else month
    day = values[0][values[0].index('/') + 1 : values[0].rindex('/')]
    day = f'0{day}' if len(day) == 1 else day
    date = f'{year}-{month}-{day}'
    confirmed_num = values[1].item()
    rows.append([date, confirmed_num])
df_numsum_confirmed = pd.DataFrame(rows, columns=['confirmed_date', 'confirmed_numsum'])

/Users/mckim/anaconda3/envs/flu_pred/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: `item` has been deprecated and will be removed in a future version
  if sys.path[0] == '':


In [5]:
# 누적 확진자를 통해 일일 확진자 구하기

df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_numsum'] - df_numsum_confirmed.shift(1)['confirmed_numsum']
df_numsum_confirmed = df_numsum_confirmed.dropna()
df_numsum_confirmed['confirmed_num'] = df_numsum_confirmed['confirmed_num'].astype(int)

In [6]:
# 일일 확진자수가 음수인 경우 그 전날 확진자를 가져다 쓰는 과정

for idx, row in df_numsum_confirmed[df_numsum_confirmed['confirmed_num'] < 0].iterrows():
    print("before", df_numsum_confirmed.loc[idx,'confirmed_num'])
    df_numsum_confirmed.loc[idx,'confirmed_num'] = df_numsum_confirmed.loc[idx-1,'confirmed_num']
    print("after", df_numsum_confirmed.loc[idx,'confirmed_num'])
    print()

before -28
after 28

before -53
after 34



In [7]:
df_numsum_confirmed.tail(5)

,confirmed_date,confirmed_numsum,confirmed_num
589,2021-09-02,1096881,13900
590,2021-09-03,1104971,8090
591,2021-09-04,1112964,7993
592,2021-09-05,1117596,4632
593,2021-09-06,1117596,0


In [1]:
df_israel_covid = df_israel_covid[df_israel_covid['test_date'] < '2021-09-01']
df_israel_covid = df_israel_covid[df_israel_covid['test_date'] >= '2021-06-01']
df_israel_isolation_per_day = df_israel_isolation_per_day[df_israel_isolation_per_day['date'] < '2021-09-01']
df_israel_isolation_per_day = df_israel_isolation_per_day[df_israel_isolation_per_day['date'] >= '2021-06-01']
df_numsum_confirmed = df_numsum_confirmed[df_numsum_confirmed['confirmed_date'] < '2021-06-01']
df_numsum_confirmed = df_numsum_confirmed[df_numsum_confirmed['confirmed_date'] >= '2021-06-01']

NameError: name 'df_israel_covid' is not defined

In [9]:
df_israel_isolation_per_day['join_date'] = (pd.to_datetime(df_israel_isolation_per_day['date']) + timedelta(days=1)).dt.strftime('%Y-%m-%d')

# 요일을 숫자로 기록한 컬럼 하나 만듬
df_israel_covid['weekday'] = pd.to_datetime(df_israel_covid['test_date']).dt.dayofweek

In [10]:
# 기존 이스라엘 데이터에 격리자 컬럼을 넣어주는 과정

df_israel_covid = pd.merge(df_israel_covid, df_israel_isolation_per_day, how='left', left_on='test_date', right_on='join_date')

In [11]:
df_israel_covid = df_israel_covid.sort_values('test_date')
df_israel_covid.head(5)

,Unnamed: 0,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date
0,1183158,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
22,1182576,2021-06-01,0,0,0,0,0,אחר,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
23,1180371,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
24,1181561,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN
25,1179102,2021-06-01,0,0,0,0,0,שלילי,NaN,NaN,Other,1,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# 기존 이스라엘 데이터에 일일 확진자랑 일일 누적 확진자 컬럼을 넣어주는 과정

df_israel_covid = pd.merge(df_israel_covid, df_numsum_confirmed, how='left', left_on='test_date', right_on='confirmed_date')

In [13]:
df_israel_covid.shape[0]

11602

In [14]:
head_ne = df_israel_covid[df_israel_covid['head_ache'] == 0]
head_po = df_israel_covid[df_israel_covid['head_ache'] == 1]
print(head_ne.shape[0]+head_po.shape[0])

11602


## 번역하기

In [15]:
def gender_map(gender):
    if gender == 'נקבה': return 1 #'Female'
    elif gender == 'זכר': return 0 #'Male'
    return -1

In [16]:
def corona_result_map(corona_result):
    if corona_result == 'אחר': return -1 # 'Other'
    elif corona_result == 'שלילי': return 0 # 'Negative'
    elif corona_result == 'חיובי': return 1 # 'Positive'
    return ''

In [17]:
def age_60_and_above_map(age_60_and_above):
    if age_60_and_above == 'Yes': return 1
    elif age_60_and_above == 'No': return 0
    return -1

In [18]:
def test_indication_map(test_indication):
    if test_indication == 'Other': return 0
    elif test_indication == 'Abroad': return 1
    elif  test_indication == 'Contact with confirmed': return 2
    else: return -1

In [19]:
df_israel_covid['gender'] = df_israel_covid['gender'].apply(gender_map)
df_israel_covid['corona_result'] = df_israel_covid['corona_result'].apply(corona_result_map)
df_israel_covid['age_60_and_above'] = df_israel_covid['age_60_and_above'].apply(age_60_and_above_map)
df_israel_covid['test_indication'] = df_israel_covid['test_indication'].apply(test_indication_map)

## Data 기초 통계

In [20]:
# 양성 데이터 수
num_of_pos = df_israel_covid[df_israel_covid['corona_result'] == 1].shape[0]

In [21]:
num_of_pos

554

In [22]:
# 음성 데이터 수
num_of_neg = df_israel_covid[df_israel_covid['corona_result'] == 0].shape[0]

In [23]:
num_of_neg

10529

In [24]:
# other 데이터 수
num_of_oth = df_israel_covid[df_israel_covid['corona_result'] == -1].shape[0]

In [25]:
num_of_oth

519

In [26]:
print(f"양성/음성 비율: {round(num_of_pos/num_of_neg, 2)}")

양성/음성 비율: 0.05


In [27]:
total_num = num_of_pos + num_of_neg + num_of_oth
total_num

11602

## 증상 유, 증상 무, 증상갯수/전체갯수, 전체 증상 갯수

In [28]:
num_cough = df_israel_covid[df_israel_covid['cough'] == 1].shape[0]
num_not_cough = df_israel_covid[df_israel_covid['cough'] == 0].shape[0]
print(num_cough, num_not_cough, num_cough/total_num, num_cough+num_not_cough)

159 11443 0.01370453370108602 11602


In [29]:
num_fever = df_israel_covid[df_israel_covid['fever'] == 1].shape[0]
num_not_fever = df_israel_covid[df_israel_covid['fever'] == 0].shape[0]
print(num_fever, num_not_fever, num_fever/total_num, num_fever+num_not_fever)

155 11447 0.013359765557662473 11602


In [30]:
num_sore_throat = df_israel_covid[df_israel_covid['sore_throat'] == 1].shape[0]
num_not_sore_throat = df_israel_covid[df_israel_covid['sore_throat'] == 0].shape[0]
print(num_sore_throat, num_not_sore_throat, num_sore_throat/total_num, num_sore_throat+num_not_sore_throat)

73 11529 0.006292018617479745 11602


In [31]:
num_shortness_of_breath = df_israel_covid[df_israel_covid['shortness_of_breath'] == 1].shape[0]
num_not_shortness_of_breath = df_israel_covid[df_israel_covid['shortness_of_breath'] == 0].shape[0]
print(num_shortness_of_breath, num_not_shortness_of_breath, num_shortness_of_breath/total_num, num_shortness_of_breath+num_not_shortness_of_breath)

11 11591 0.000948112394414756 11602


In [32]:
num_head_ache = df_israel_covid[df_israel_covid['head_ache'] == 1].shape[0]
num_not_head_ache = df_israel_covid[df_israel_covid['head_ache'] == 0].shape[0]
print(num_head_ache, num_not_head_ache, num_head_ache/total_num, num_head_ache+num_not_head_ache)

118 11484 0.010170660230994655 11602


In [33]:
num_women = df_israel_covid[df_israel_covid['gender'] == 1].shape[0]
num_man = df_israel_covid[df_israel_covid['gender'] == 0].shape[0]
num_unk = df_israel_covid[df_israel_covid['gender'] == -1].shape[0]
print(num_women, num_man, num_unk, num_man/(num_women+num_man), num_women+num_man+num_unk)

5036 4333 2233 0.46248265556622903 11602


In [34]:
num_60_up = df_israel_covid[df_israel_covid['age_60_and_above'] == 1].shape[0]
num_60_down = df_israel_covid[df_israel_covid['age_60_and_above'] == 0].shape[0]
num_unk = df_israel_covid[df_israel_covid['age_60_and_above'] == -1].shape[0]
print(num_60_up, num_60_down, num_unk,  num_60_up/(num_60_up+num_60_down+num_unk), num_60_up+num_60_down+num_unk)

1240 6867 3495 0.10687812446129978 11602


In [35]:
num_other = df_israel_covid[df_israel_covid['test_indication'] == 0].shape[0]
num_abroad = df_israel_covid[df_israel_covid['test_indication'] == 1].shape[0]
num_contact = df_israel_covid[df_israel_covid['test_indication'] == 2].shape[0]
# num_unk = df_israel_covid[df_israel_covid['test_indication'] == -1].shape[0]
print(num_other, num_abroad, num_contact, num_other+num_abroad+num_contact)

11388 18 196 11602


## data를 코로나 결과가 양성, 음성, 기타인 데이터로 분리해서 비교 

In [36]:
df_israel_covid_p = df_israel_covid[df_israel_covid['corona_result'] == 1]

In [37]:
df_israel_covid_n = df_israel_covid[df_israel_covid['corona_result'] == 0]

In [38]:
df_israel_covid_o = df_israel_covid[df_israel_covid['corona_result'] == -1]

In [39]:
df_israel_covid_p.tail(5)

,Unnamed: 0,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,...,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date,confirmed_date,confirmed_numsum,confirmed_num
11539,52849,2021-08-31,0,0,0,0,0,1,0,1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11541,52816,2021-08-31,0,1,1,0,0,1,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11555,52892,2021-08-31,0,1,0,0,0,1,1,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11565,41565,2021-08-31,0,0,0,0,0,1,-1,-1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11584,41086,2021-08-31,0,1,0,0,0,1,0,1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN


In [40]:
df_israel_covid_n.tail(5)

,Unnamed: 0,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,...,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date,confirmed_date,confirmed_numsum,confirmed_num
11597,50510,2021-08-31,0,0,0,0,0,0,-1,1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11598,48997,2021-08-31,0,0,0,0,0,0,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11599,49597,2021-08-31,0,0,0,0,0,0,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11600,48016,2021-08-31,0,0,0,0,0,0,1,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11601,61288,2021-08-31,0,0,0,0,0,0,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN


In [41]:
df_israel_covid_o.tail(5)

,Unnamed: 0,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,...,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date,confirmed_date,confirmed_numsum,confirmed_num
11554,49141,2021-08-31,0,0,0,0,0,-1,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11559,42429,2021-08-31,0,0,0,0,0,-1,0,-1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11578,60150,2021-08-31,0,0,0,0,0,-1,0,1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11579,57586,2021-08-31,0,0,0,0,0,-1,1,1,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN
11590,57632,2021-08-31,0,0,0,0,0,-1,0,0,...,1,2021-08-30,30785.0,76593.0,6446,8006,2021-08-31,NaN,NaN,NaN


In [42]:
# 데이터 갯수 & 증상 비율들

def cnt_has_symptom(df):
    print('# of data ]', df.shape[0])
    print('# of fever ] ', df[df['fever'] == 1].shape[0] / df.shape[0])
    print('# of cough ] ', df[df['cough'] == 1].shape[0] / df.shape[0])    
    print('# of sore_throat ] ', df[df['sore_throat'] == 1].shape[0] / df.shape[0])    
    print('# of shortness_of_breath ] ', df[df['shortness_of_breath'] == 1].shape[0] / df.shape[0])
    print('# of head_ache ] ', df[df['head_ache'] == 1].shape[0] / df.shape[0])

In [43]:
cnt_has_symptom(df_israel_covid_p)

# of data ] 554
# of fever ]  0.17870036101083034
# of cough ]  0.17689530685920576
# of sore_throat ]  0.07220216606498195
# of shortness_of_breath ]  0.018050541516245487
# of head_ache ]  0.11913357400722022


In [44]:
cnt_has_symptom(df_israel_covid_n)

# of data ] 10529
# of fever ]  0.004463861715262608
# of cough ]  0.004843764839965809
# of sore_throat ]  0.002374394529395004
# of shortness_of_breath ]  9.497578117580017e-05
# of head_ache ]  0.003988982809383607


In [45]:
cnt_has_symptom(df_israel_covid_o)

# of data ] 519
# of fever ]  0.017341040462427744
# of cough ]  0.019267822736030827
# of sore_throat ]  0.015414258188824663
# of shortness_of_breath ]  0.0
# of head_ache ]  0.019267822736030827


## 양성 음성에 따른 증상 갯수 차이 비교

In [46]:
pos_list = [0] * 6
for idx, row in df_israel_covid_p.iterrows():
    cnt_idx = row['cough'] + row['fever'] + row['sore_throat'] + \
            + row['shortness_of_breath'] + row['head_ache']
    pos_list[cnt_idx] += 1
print(pos_list)

[376, 83, 63, 25, 6, 1]


In [47]:
print(234848/428599, 102525/428599, 62485/428599, 22067/428599, 5775/428599, 899/428599)

0.5479434156402605 0.23920961084836875 0.14578895424394364 0.05148635437786836 0.013474133164099776 0.0020975317254589955


In [48]:
neg_list = [0] * 6
for idx, row in df_israel_covid_n.iterrows():
    cnt_idx = row['cough'] + row['fever'] + row['sore_throat'] + \
            + row['shortness_of_breath'] + row['head_ache']
    neg_list[cnt_idx] += 1
print(neg_list)

[10432, 46, 34, 16, 1, 0]


In [49]:
print(5194994/5454805, 153135/5454805, 78213/5454805, 21996/5454805, 5610/5454805, 857/5454805)

0.9523702497156177 0.028073414173375583 0.01433836773266872 0.004032408124580072 0.0010284510628702584 0.00015710919088766692


## rulebasemodel preprocessing

In [55]:
df_rulebase = pd.read_excel('rulebasemodel_chart.xlsx', engine='openpyxl')

In [56]:
df_rulebase.head(5)

,cough,fever,sore_throat,shortness_of_breath,head_ache,sex,result,cough_2,fever_2,sore_throat_2,...,sex_2,result_2,Unnamed: 14,Unnamed: 15,Unnamed: 16,sex,"male: 0, female:1, dontknow -1",Unnamed: 19,Unnamed: 20,"설문에는 있는데, 남녀 차이가 없이 결과가 나온다"
0,0,0,0,0,0,-1,0,0,0,0,...,-1,0,NaN,NaN,NaN,result,"LOW: 0, MEDIUM: 1, HIGH=2 VERY LOW:3",NaN,NaN,NaN
1,1,0,0,0,0,-1,1,1,0,0,...,-1,0,NaN,NaN,NaN,test_indication,contact:1 non contact 0,NaN,NaN,contact이면 무조건 HIGH 나와서 제외
2,0,1,0,0,0,-1,1,0,1,0,...,-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,1,0,0,-1,1,0,0,1,...,-1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,0,1,0,-1,1,0,0,0,...,-1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df_symtoma_rule = df_rulebase[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'result']]
df_shine1_rule  = df_rulebase[['cough_2', 'fever_2', 'sore_throat_2', 'shortness_of_breath_2', 'head_ache_2', 'result_2']]

In [58]:
df_shine1_rule = df_shine1_rule.rename(columns={'cough_2': 'cough', 'fever_2': 'fever', 'sore_throat_2': 'sore_throat', 'shortness_of_breath_2': 'shortness_of_breath', 'head_ache_2': 'head_ache', 'result_2': 'result'})

In [59]:
display(df_symtoma_rule.head(5))
display(df_shine1_rule.head(5))

,cough,fever,sore_throat,shortness_of_breath,head_ache,result
0,0,0,0,0,0,0
1,1,0,0,0,0,1
2,0,1,0,0,0,1
3,0,0,1,0,0,1
4,0,0,0,1,0,1


,cough,fever,sore_throat,shortness_of_breath,head_ache,result
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,0,1,0,0,1
4,0,0,0,1,0,2


## model input preprocessing

In [60]:
# other 빼버림
df_israel_covid_without_o = df_israel_covid[df_israel_covid['corona_result'] != -1]

In [61]:
df_israel_covid_without_o.shape[0]

11083

In [62]:
# 성별 없는 사람들 빼버림
df_israel_covid_without_o = df_israel_covid_without_o[df_israel_covid_without_o['gender'] != -1]

In [63]:
df_israel_covid_without_o.shape[0]

8963

In [64]:
df_israel_covid_without_o = df_israel_covid_without_o.sort_values(['test_date']).reset_index(drop=True)

In [65]:
df_israel_covid_without_o['test_date'].min()

'2021-06-05'

In [66]:
# # week_num 이라는 칼럼을 새로 만들었다. 1년에 52주라 
# def date_to_weeknum_map(test_date):
#     week_num_st_idx = 52
#     year_last_num = int(test_date[2:4])
#     week_num = datetime.date(int(test_date[:4]), int(test_date[5:7]), int(test_date[8:])).isocalendar()[1]
#     return week_num_st_idx * year_last_num + week_num

In [67]:
# df_israel_covid_without_o['week_num'] = df_israel_covid_without_o['test_date'].apply(date_to_weeknum_map)

## 확진자 수 가져오는 api (지금은 사용x)

In [68]:
# from urllib.request import Request, urlopen
# from urllib.parse import urlencode, quote_plus
# from bs4 import BeautifulSoup

# url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19NatInfStateJson'
# service_key = 'KYLVjDwXBWqEgO6Ph8q7a2oIjeVFvYkaLsmUBEa2qIO7YqtQYm4aUpsVIPWomkpPDfeJXNkITA4OBlgMip2v7g%3D%3D'
# queryParams = '?' + urlencode({ quote_plus('serviceKey') : service_key, quote_plus('pageNo') : '1', quote_plus('numOfRows') : '10', quote_plus('startCreateDt') : '20200310', quote_plus('endCreateDt') : '20210812' })
# queryParams = queryParams.replace('%25', '%')
# print(url + queryParams)
# request = Request(url + queryParams)
# request.get_method = lambda: 'GET'
# response_body = urlopen(request).read()

# bs = BeautifulSoup(response_body, 'html.parser')

# links = bs.find_all('item')
# # print(links)
# for link in links:
#      if link.find_all('nationnmen', text=re.compile("Israel")):
#         print(link.find('nationnmen').text)
#         print(link.find('stdday').text)
#         print(link.find('natdefcnt').text)

In [69]:
df_israel_covid_without_o[(df_israel_covid_without_o['test_date'] == '2021-01-01') & (df_israel_covid_without_o['corona_result'] == 1)]

,Unnamed: 0,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,...,weekday,date,isolated_today_contact_with_confirmed,isolated_today_abroad,new_contact_with_confirmed,new_from_abroad,join_date,confirmed_date,confirmed_numsum,confirmed_num


In [70]:
df_israel_covid_without_o['new_contact_with_confirmed'] = df_israel_covid_without_o['new_contact_with_confirmed'].astype(int)

In [71]:
df_israel_covid_without_o['contact'] = df_israel_covid_without_o['new_contact_with_confirmed'] / 36903

In [72]:
df_confirm_by_sampling = df_israel_covid_without_o[df_israel_covid_without_o['corona_result'] == 1].groupby('test_date').count().reset_index()[['test_date', 'date']]
df_confirm_by_sampling = df_confirm_by_sampling.rename(columns={'test_date': 'yesterday', 'date': 'positive_cnt'})

In [73]:
df_israel_covid_without_o = pd.merge(df_israel_covid_without_o, df_confirm_by_sampling, how='left', left_on='date', right_on='yesterday')

In [74]:
df_israel_covid_without_o.shape[0]

8963

In [75]:
df_israel_covid_without_o = df_israel_covid_without_o[~df_israel_covid_without_o['positive_cnt'].isnull()]

In [76]:
df_israel_covid_without_o['positive_cnt'] = df_israel_covid_without_o['positive_cnt'].astype(int)

In [77]:
df_israel_covid_without_o['positive_ratio'] = df_israel_covid_without_o['positive_cnt'] / df_israel_covid_without_o['positive_cnt'].max()

In [78]:
df_israel_covid_without_o['weekday_ratio'] = df_israel_covid_without_o['weekday'] / 6

In [79]:
df_israel_covid_without_o['confirmed_ratio'] = df_israel_covid_without_o['confirmed_num'] / df_israel_covid_without_o['confirmed_num'].max()

In [80]:
df_israel_covid_without_o.columns

Index(['Unnamed: 0', 'test_date', 'cough', 'fever', 'sore_throat',
       'shortness_of_breath', 'head_ache', 'corona_result', 'age_60_and_above',
       'gender', 'test_indication', 'weekday', 'date',
       'isolated_today_contact_with_confirmed', 'isolated_today_abroad',
       'new_contact_with_confirmed', 'new_from_abroad', 'join_date',
       'confirmed_date', 'confirmed_numsum', 'confirmed_num', 'contact',
       'yesterday', 'positive_cnt', 'positive_ratio', 'weekday_ratio',
       'confirmed_ratio'],
      dtype='object')

In [81]:
# 여기서 내가 사용할 feature 고르기
raw_datas = df_israel_covid_without_o[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'corona_result']].values
# raw_datas = df_israel_covid_without_o[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'age_60_and_above', 'gender', 'confirmed_ratio', 'weekday_ratio', 'corona_result']].values

In [82]:
input_datas = raw_datas[:, :-1]
gt_datas = raw_datas[:, -1]

In [83]:
input_datas.shape

(7230, 5)

In [118]:
def get_result_by_rule(input_data, df_rule):
    cough_flag = df_rule['cough'] == input_data[0]
    fever_flag = df_rule['fever'] == input_data[1]
    sore_throat_flag = df_rule['sore_throat'] == input_data[2]
    shortness_of_breath_flag = df_rule['shortness_of_breath'] == input_data[3]
    head_ache_flag = df_rule['head_ache'] == input_data[4]
    
    final_flag = cough_flag & fever_flag & sore_throat_flag & shortness_of_breath_flag & head_ache_flag
    
    return df_rule[final_flag]['result'].item()

In [125]:
from collections import defaultdict
def rulebase_classifier(input_data, dict_rule):
    result_by_rule = dict_rule[tuple(input_data)]
    dict_return_value = defaultdict(int)
    dict_return_value.update({
        0: 0,
        1: 0.3,
        2: 0.8,
    })
    return dict_return_value[result_by_rule]
def get_preds_by_rule(input_datas, dict_rule):
    preds = []
    for idx, input_data in enumerate(input_datas):
        preds.append(rulebase_classifier(input_data, dict_rule))
#         preds = [rulebase_classifier(input_data, df_rule) for input_data in input_datas]
    return np.array(preds, dtype=np.float64)

dict_rule = df_shine1_rule.set_index(['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']).to_dict()['result']
dict_rule

In [132]:
preds = get_preds_by_rule(input_datas, dict_rule)

In [133]:
sum(preds), sum(gt_datas)

(53.499999999999865, 523)

In [134]:
roc_auc_score(preds, gt_datas)

ValueError: continuous format is not supported

In [ ]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data)).astype(int)
    test_set_size = int(len(data) * test_ratio)
    
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    
    print(len(data))
    print(test_indices.dtype)
    print(train_indices.dtype)
    
    test_data = data[:test_set_size]
    train_data = data[test_set_size:]
    test_data = data[test_indices]
    train_data = data[train_indices]
    
    return train_data, test_data

In [ ]:
def split_input_gt(non_split_data):
    splited_input = non_split_data[:, :-1]
    splited_gt = non_split_data[:, -1]
    
    return splited_input, splited_gt

In [ ]:
not_test_datas, test_datas = split_train_test(raw_datas, test_ratio=0.2)
train_datas, valid_datas = split_train_test(not_test_datas, test_ratio=0.2)

train_input, train_gt = split_input_gt(train_datas)
valid_input, valid_gt = split_input_gt(valid_datas)
test_input, test_gt = split_input_gt(test_datas)

In [ ]:
print(train_input.shape, train_gt.shape, valid_input.shape, valid_gt.shape, test_input.shape, test_gt.shape)

In [ ]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

In [ ]:
class IsraelDataset(Dataset):
    def __init__(self, input_data, gt, transform=None, target_transform=None):
        self.input_data = input_data
        self.gt = gt
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, idx):
        return self.input_data[idx], self.gt[idx]

In [ ]:
# def plot_roc_curve(fper, tper):
#     plt.plot(fper, tper, color='red', label='ROC')
#     plt.plot([0, 1], [0, 1], color='green', linestyle='--')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('Receiver Operating Characteristic Curve')
#     plt.legend()
#     plt.show()

In [ ]:
def display_performance(clf, pred, valid_input, valid_gt):
    pred = np.array(pred)

    print('accuracy %.4f' % (accuracy_score(pred, valid_gt)))
    print('precision %.4f' % (precision_score(pred, valid_gt)))
    print('recall %.4f' % (recall_score(pred, valid_gt)))
    print('f1-score %.4f' % (f1_score(pred, valid_gt)))
    print(confusion_matrix(pred, valid_gt))
    plot_roc_curve(clf, valid_input, valid_gt)

In [ ]:
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

logistic_clf = LogisticRegression(C=1000.0, random_state=0)
logistic_clf = logistic_clf.fit(train_input, train_gt)

In [ ]:
pred = logistic_clf.predict(valid_input)
display_performance(logistic_clf, pred, valid_input, valid_gt)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=5, random_state=0)
rf_clf.fit(train_input, train_gt)

In [ ]:
pred = rf_clf.predict(valid_input)
display_performance(logistic_clf, pred, valid_input, valid_gt)

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(train_input, train_gt)

In [ ]:
pred = dt_clf.predict(valid_input)
display_performance(logistic_clf, pred, valid_input, valid_gt)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(train_input, train_gt)

In [ ]:
pred = gb_clf.predict(valid_input)
display_performance(logistic_clf, pred, valid_input, valid_gt)

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()
#         nn.BatchNorm1d(9)
        self.fcs1 = nn.Sequential(
            nn.Linear(9, 400),
            nn.BatchNorm1d(400),
            nn.ReLU()
        )
#         dropout 넣기
#         nn.linear, L1 norm L2 norm
        self.fcs2 = nn.Sequential(
            nn.Linear(400, 400),
            nn.BatchNorm1d(400),
            nn.ReLU()
        )
        self.fcs3 = nn.Sequential(
            nn.Linear(400, 400),
            nn.BatchNorm1d(400),
            nn.ReLU()
        )
        self.fcs4 = nn.Sequential(
            nn.Linear(400, 400),
            nn.BatchNorm1d(400),
            nn.ReLU()
        )
        self.fcs5 = nn.Sequential(
            nn.Linear(400, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = self.fcs1(x)
        x = x + self.fcs2(x)
        x = x + self.fcs3(x)
        x = x + self.fcs4(x)
        pred = self.fcs5(x)
        return pred

In [ ]:
training_data = IsraelDataset(torch.Tensor(train_input), torch.Tensor(train_gt.reshape(-1, 1)))
valid_data = IsraelDataset(torch.Tensor(valid_input), torch.Tensor(valid_gt.reshape(-1, 1)))

train_dataloader = DataLoader(training_data, batch_size = 128, shuffle=False)
valid_dataloader = DataLoader(valid_data, batch_size=1, shuffle=False)

valid_input.shape

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
model = BaselineModel().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # lr 크기좀 늘려보기

In [ ]:
# train_running_loss = 0.0
# num_epochs = 20

# for epoch in tqdm(range(num_epochs)):
#     model.train()
#     for train_data in tqdm(train_dataloader):
#         train_inputs, train_labels = train_data
#         train_inputs = train_inputs.to(device)
#         train_labels = train_labels.to(device)

#         optimizer.zero_grad()
        
#         train_outputs = model(train_inputs)
#         train_loss = criterion(train_outputs, train_labels)
#         train_loss.backward()
#         optimizer.step()
    
#     model.eval()

#     pred = []
#     positive_res_data = []
#     negative_res_data = []
#     for valid_data in tqdm(valid_dataloader):
#         valid_inputs, train_labels = valid_data
#         valid_outputs = model(torch.Tensor(valid_inputs).to(device))
#         if valid_outputs[0][0].item() > 0.5:
#             pred.append([valid_outputs[0][0].item()])
#             positive_res_data.append(valid_inputs.numpy().tolist())
#         else:
#             pred.append([valid_outputs[0][0].item()])
#             negative_res_data.append(valid_inputs.numpy().tolist())
            
#     display_performance(pred, valid_gt)

#     print('Done')
# display_performance(model, pred, valid_input, valid_gt)

In [ ]:
# model.eval()

# pred = []
# positive_res_data = []
# negative_res_data = []
# for valid_data in tqdm(valid_dataloader):
        
#     valid_inputs, train_labels = valid_data
#     valid_outputs = model(torch.Tensor(valid_inputs).to(device))
#     if valid_outputs[0][0].item() > 0.5:
#         pred.append([1])
#         positive_res_data.append(valid_inputs.numpy().tolist())
#     else:
#         pred.append([0])
#         negative_res_data.append(valid_inputs.numpy().tolist())

In [ ]:
# def display_performance(pred, valid_gt):
#     pred = np.array(pred)

#     print('accuracy %.4f' % (accuracy_score(pred, valid_gt)))
#     print('precision %.4f' % (precision_score(pred, valid_gt)))
#     print('recall %.4f' % (recall_score(pred, valid_gt)))
#     print('f1-score %.4f' % (f1_score(pred, valid_gt)))
#     print(confusion_matrix(pred, valid_gt))
#     fper, tper, thresholds = roc_curve(pred, valid_gt)
#     print(tper)
#     print(fper)
#     print(thresholds)
#     print(roc_auc_score(valid_gt, pred))
#     optimal_idx = np.argmax(tper - fper)
#     optimal_threshold = thresholds[optimal_idx]
#     print("Threshold value is:", optimal_threshold)
#     # plot_roc_curve(fpr, tpr)
#     plot_roc_curve(fper, tper)

### 8-feature - epoch 1
- accuracy 0.8779
- precision 0.2652
- recall 0.4887
- f1-score 0.3438

### 8-feature - epoch 10
- accuracy 0.1348
- precision 0.9928
- recall 0.1217
- f1-score 0.2169

### 9-feature(isolated_today_contact_with_confirmed or new_contact_with_confirmed)
- accuracy 0.8793
- precision 0.0000
- 모두 음성으로 판단하는 대참사

### 9-feature(isolated_today_contact_with_confirmed or new_contact_with_confirmed) - epoch 5
- accuracy 0.8699
- precision 0.5792
- recall 0.4685
- f1-score 0.5180

### 9-feature(contact) - epoch 20
- accuracy 0.8702
- precision 0.2208
- recall 0.4144
- f1-score 0.2881

### 9-feature(positive ratio) - epoch 20
- accuracy 0.7904
- precision 0.5766
- recall 0.3010
- f1-score 0.3956

### 10-feature( 'positive_ratio', 'contact') - epoch 20
- accuracy 0.8713
- precision 0.5797
- recall 0.4671
- f1-score 0.5174

### 12-feature('contact', 'positive_ratio', 'weekday_ratio', 'confirmed_ratio') - epoch 20
- accuracy 0.8698
- precision 0.5809
- recall 0.4650
- f1-score 0.5166


accuracy 0.8793
precision 0.1684
recall 0.4877
f1-score 0.2503
[[740307  85788]
 [ 18244  17368]]

In [ ]:
df_pos = pd.DataFrame(np.array(positive_res_data).reshape(-1, 8), columns=['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'age_60_and_above', 'gender', 'test_indication'])
df_neg = pd.DataFrame(np.array(negative_res_data).reshape(-1, 8), columns=['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 'age_60_and_above', 'gender', 'test_indication'])

In [ ]:
print(f"양성 갯수: {df_pos.shape[0]}, 음성 갯수: {df_neg.shape[0]}, 양성 / 음성 비율: {round(df_pos.shape[0] / df_neg.shape[0] * 100, 2)}%")

In [ ]:
df_pos_symptom_cnt = [0] * 6
df_neg_symptom_cnt = [0] * 6
pos_symptom_confusion_matrix = np.zeros((2,2,2,2,2), dtype=int)
neg_symptom_confusion_matrix = np.zeros((2,2,2,2,2), dtype=int)

for idx, row in tqdm(df_pos.iterrows()):
    symptoms = row[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']].astype(int).tolist()
    pos_symptom_confusion_matrix[symptoms[0], symptoms[1], symptoms[2], symptoms[3], symptoms[4]] += 1
    cnt_idx = int(sum(symptoms))
    df_pos_symptom_cnt[cnt_idx] += 1
    
for idx, row in tqdm(df_neg.iterrows()):
    #     if idx == df_pos.shape[0]: break
    symptoms = row[['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache']].astype(int).tolist()
    neg_symptom_confusion_matrix[symptoms[0], symptoms[1], symptoms[2], symptoms[3], symptoms[4]] += 1
    cnt_idx = int(sum(symptoms))
    df_neg_symptom_cnt[cnt_idx] += 1

In [ ]:
df_pos.shape[0], df_neg.shape[0]

In [ ]:
def positive_percent_of_ormore_number(number):
    neg_ormore_num = sum(df_neg_symptom_cnt[number:])
    pos_ormore_num = sum(df_pos_symptom_cnt[number:])
    total = neg_ormore_num + pos_ormore_num
    if total == 0: 'lower the number'
    return round((pos_ormore_num / total) * 100, 2)

In [ ]:
print(f"음성 예측 데이터 [0개, 1개, 2개, 3개, 4개, 5개] : {df_neg_symptom_cnt[:6]}")
print(f"양성 예측 데이터 [0개, 1개, 2개, 3개, 4개, 5개] : {df_pos_symptom_cnt[:6]}\n")

for number in range(6):
    print(f"{number}개 이상] {positive_percent_of_ormore_number(number)}% 양성")
    print(f"증상 {number}개] 음성/양성% : {round(df_neg_symptom_cnt[number] / df_pos_symptom_cnt[number]*100, 2)}%\n")

In [ ]:
pos_symptom_confusion_matrix

In [ ]:
def confusion_matrix_display(confusion_matrix):
    for i in range(32):
        idx = bin(i).replace("0b", "0" * (7-len(bin(i))))
        cnt = 0
        if idx[0] == '1': 
            print('[ cough',end = ' ')
            cnt += 1
        if idx[1] == '1': 
            print('fever',end = ' ')
            cnt += 1
        if idx[2] == '1': 
            print('sore_throat',end = ' ')
            cnt += 1
        if idx[3] == '1': 
            print('shortness_of_breath',end = ' ')
            cnt += 1
        if idx[4] == '1': 
            print('head_ache',end = ' ')
            cnt += 1
        print(f'{cnt}개 증상 ]')
            
        print(confusion_matrix[int(idx[0]),int(idx[1]),int(idx[2]),int(idx[3]),int(idx[4])])

In [ ]:
confusion_matrix_display(neg_symptom_confusion_matrix)

In [ ]:
confusion_matrix_display(pos_symptom_confusion_matrix)

In [ ]:
pred = np.array(pred)

print('accuracy %.4f' % (accuracy_score(pred, valid_gt)))
print('precision %.4f' % (precision_score(pred, valid_gt)))
print('recall %.4f' % (recall_score(pred, valid_gt)))
print('f1-score %.4f' % (f1_score(pred, valid_gt)))
print(confusion_matrix(pred, valid_gt))

In [ ]:
pred
# pred_chk

### random forest result
- accuracy 0.9257
- precision 0.2314
- recall 0.5553
- f1-score 0.3267
- [[791936  52206]
- [ 12591  15720]]

### 1-layer sigmoid epoch3 result
- accuracy 0.9222
- precision 0.1958
- recall 0.5013
- f1-score 0.2816
- [[791299  54627]
- [ 13228  13299]]

### compare with logistic regression
- accuracy 0.9224
- precision 0.1953
- recall 0.5049
- f1-score 0.2816
- [[791517  54661]
- [ 13010  13265]]

### 2-layer epoch3 result
- 8-4-1(sig-sig)
- accuracy 0.9248
- precision 0.2626
- recall 0.5342
- f1-score 0.3521
- [[788976  50090]
-  [ 15551  17836]]

### 2-layer epoch3 result
- 8-4-1(relu-sig)
- accuracy 0.9256 
- precision 0.1999
- recall 0.5635
- f1-score 0.2951
- [[794008  54349]
- [ 10519  13577]]

### 2-layer epoch3 result
- 8-20-1(relu-sig)
- accuracy 0.9255
- precision 0.2392
- recall 0.5498
- f1-score 0.3333
- [[791226  51680]
-  [ 13301  16246]]

### 2-layer epcoh3 result 
- 8-400-1(relu-sig)
- accuracy 0.9256
- precision 0.2904
- recall 0.5415
- f1-score 0.3781
- [[787821  48197]
- [ 16706  19729]]

### 3-layer epoch3 result
- 8-4-2-1 (relu-relu-sigmoid) (sigmoid-sigmoid-sigmoid 동일한 결과)
- accuracy 0.9221
- precision 0.0000
- recall 0.0000
- f1-score 0.0000
- [[804527  67926]
-  [     0      0]]
- 확인 결과 모든 출력을 음성으로 출력함(1-layer가 오히려 낫네?)

## 시간 순서대로 train, valid, test 나눔

### Logistic Regression
- accuracy 0.8782
- precision 0.2240
- recall 0.4901
- f1-score 0.3075
- [[742649  81690]
  [ 24533  23581]]
  
### Random Forest
- accuracy 0.8761
- precision 0.4298
- recall 0.4848
- f1-score 0.4556
- [[719101  60026]
- [ 48081  45245]]

### Decision Tree
- accuracy 0.8761
- precision 0.4299
- recall 0.4848
- f1-score 0.4557
- [[719094  60017]
- [ 48088  45254]]

### Gradient Boosting 
- accuracy 0.8789
- precision 0.3059
- recall 0.4973
- f1-score 0.3788
- [[734633  73070]
- [ 32549  32201]]

### 3-layer NN 
- epochs = 10, batch_size = 128
- 8-1200-1
- accuracy 0.8699
- precision 0.5794
- recall 0.4683
- f1-score 0.5180
- [[697921  44274]
- [ 69261  60997]]